# Clean Data

- Author: Wen Xin
- Description: This file is used to clean data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")

In [2]:
data=pd.read_csv('all_data.csv')

In [3]:
data.head()

,user_id,order_sd,active_days,lastgap,avg_gap,min_gap,max_gap,total_orders,w1_total_orders,w2_total_orders,...,last_distance,order_nums,city_id,sex,age,occupation,invite_id,invite_code,login_platform,app_version
0,100002020,0.663138,10,3.0,4.888889,1.0,12.0,16.0,4.0,2.0,...,56.0,8.0,0,1,80,互联网,0,9828.0,Android,2.8.2
1,100002033,0.602960,18,7.0,2.941176,1.0,11.0,22.0,8.0,5.0,...,2099.0,10.0,0,0,NaN,NaN,0,6126.0,iOS,3.5.0
2,100002035,0.391684,13,5.0,4.500000,2.0,12.0,13.0,3.0,2.0,...,1423.0,7.0,0,0,NaN,NaN,0,3227.0,iOS,2.4.6
3,100002036,0.337142,9,5.0,8.625000,1.0,53.0,9.0,2.0,0.0,...,1876.0,5.0,0,0,NaN,NaN,0,3844.0,iOS,3.4.0
4,100002040,0.535490,12,5.0,3.454545,1.0,6.0,15.0,0.0,0.0,...,2140.0,2.0,0,2,90,上班族,0,4316.0,Android,3.2.1


In [4]:
data=data.drop_duplicates(keep='first')

In [5]:
data.to_csv('all_data_corrected.csv',index=False)

In [6]:
#1.替换错误的last coupon fee
#  转换last coupon fee的格式 string->int
data['last_coupon_fee'].replace('\\N','0.0',inplace=True)
data['last_coupon_fee']=data['last_coupon_fee'].astype(float)

In [7]:
#2.替换错误的年龄
data['age']=data['age'].astype(str)
data['age'].unique()

array(['80.0', 'nan', '90.0', '70.0', '60.0', '0.0', '90', '80', '70',
       '60', '00', '前', '0', '0后'], dtype=object)

In [8]:
data['age'].replace('nan',np.nan,inplace=True)
data['age'].replace('前',np.nan,inplace=True)
data['age'].replace('0后',np.nan,inplace=True)
data['age']=data['age'].astype(float)

In [9]:
data['age'].unique()

array([80., nan, 90., 70., 60.,  0.])

In [10]:
data['age'].replace(0.,100.,inplace=True)

In [11]:
data['age']=data['age'].fillna(-1)

In [12]:
# 3.userid==101427199，赋值order_nums=1, 反正就一个没赋值好2333
data['order_nums']=data['order_nums'].fillna(1)

In [17]:
data[pd.isnull(data['order_nums'])]

,order_sd,active_days,lastgap,avg_gap,min_gap,max_gap,total_orders,w1_total_orders,w2_total_orders,w3_total_orders,...,send2update_time_max,intime_rate,last_coupon_fee,last_send_type,order_nums,city_id,sex,age,login_platform,app_version


In [18]:
# 4.drop user_id,invite_id,occupation,一些和distance相关的feature
data.columns

Index(['order_sd', 'active_days', 'lastgap', 'avg_gap', 'min_gap', 'max_gap',
       'total_orders', 'w1_total_orders', 'w2_total_orders', 'w3_total_orders',
       'w4_total_orders', 'w5_total_orders', 'w6_total_orders',
       'w7_total_orders', 'total_couponworth', 'w1_total_couponworth',
       'w2_total_couponworth', 'w3_total_couponworth', 'w4_total_couponworth',
       'w5_total_couponworth', 'w6_total_couponworth', 'w7_total_couponworth',
       'label', 'first_order', 'last_order', 'accountLength',
       'overall_time_mean', 'update2create_time_mean', 'send2update_time_mean',
       'overall_time_min', 'update2create_time_min', 'send2update_time_min',
       'overall_time_max', 'update2create_time_max', 'send2update_time_max',
       'intime_rate', 'last_coupon_fee', 'last_send_type', 'order_nums',
       'city_id', 'sex', 'age', 'login_platform', 'app_version'],
      dtype='object')

In [19]:
data=data.drop(['user_id','invite_id','occupation',
          'distance_min', 'distance_max', 'distance_mean',
          'last_distance','invite_code'],axis=1)

KeyError: "['user_id' 'invite_id' 'occupation' 'distance_min' 'distance_max'\n 'distance_mean' 'last_distance' 'invite_code'] not found in axis"

In [20]:
# 5.login platform

In [21]:
data['login_platform'].unique()

array(['Android', 'iOS', 'mobile', nan], dtype=object)

In [22]:
def map_platform(p):
    if p == 'iOS':
        return 1
    elif p == 'Android':
        return 0
    else:
        return -1

data['login_platform_b'] = data['login_platform'].map(map_platform)

In [23]:
data['login_platform_b'].unique()

array([ 0,  1, -1])

In [24]:
# 6.app version
# 逻辑是愿意更新的人肯定app version要新，不更新的说明不用这个app了，也不会更新了

In [25]:
data['app_version'].unique()

array(['2.8.2', '3.5.0', '2.4.6', '3.4.0', '3.2.1', '2.6.0', '3.7.1',
       '3.3.0', '2.9.0', '3.3.1', '2.7.0', '3.2.0', '3.0.0', '2.6.1',
       '2.4.3', '2.7.1', '2.1.4', '2.8.1', '3.7.0', '3.1.0', '2.9.6',
       '2.5.1', '2.4.1', '2.1.0', '3.6.0', '2.2.1', '2.1.2', '2.9.7',
       '2.9.1', nan, '2.4.5', '2.3.0', '2.5.0', '2.8.0', '2.1.1', '2.4.4',
       '2.4.0', '2.0.3', '2.4.2', '2.0.0', '2.2.0'], dtype=object)

In [26]:
def map_app_versions(p):
    updated_versions=['2.8.2', '3.5.0', '3.4.0', '3.2.1', '3.7.1', \
    '3.3.0', '2.9.0', '3.3.1', '3.2.0', '3.0.0', '2.8.1', '3.7.0',\
    '3.1.0', '2.9.6', '3.6.0',  '2.9.7','2.9.1']
    if p in updated_versions:
        return 1
    else:
        return 0

data['app_version_b'] = data['app_version'].map(map_app_versions)

In [27]:
data['app_version_b'].unique()

array([1, 0])

In [28]:
data=data.drop(['login_platform','app_version'],axis=1)

In [29]:
# 7.gap fillna fill 100
data.columns

Index(['order_sd', 'active_days', 'lastgap', 'avg_gap', 'min_gap', 'max_gap',
       'total_orders', 'w1_total_orders', 'w2_total_orders', 'w3_total_orders',
       'w4_total_orders', 'w5_total_orders', 'w6_total_orders',
       'w7_total_orders', 'total_couponworth', 'w1_total_couponworth',
       'w2_total_couponworth', 'w3_total_couponworth', 'w4_total_couponworth',
       'w5_total_couponworth', 'w6_total_couponworth', 'w7_total_couponworth',
       'label', 'first_order', 'last_order', 'accountLength',
       'overall_time_mean', 'update2create_time_mean', 'send2update_time_mean',
       'overall_time_min', 'update2create_time_min', 'send2update_time_min',
       'overall_time_max', 'update2create_time_max', 'send2update_time_max',
       'intime_rate', 'last_coupon_fee', 'last_send_type', 'order_nums',
       'city_id', 'sex', 'age', 'login_platform_b', 'app_version_b'],
      dtype='object')

In [30]:
data['lastgap']=data['lastgap'].fillna(100)
data['avg_gap']=data['avg_gap'].fillna(100)
data['min_gap']=data['min_gap'].fillna(100)
data['max_gap']=data['max_gap'].fillna(100)

In [31]:
# 8.所有time相关的fillna fill 500000
time_features=['overall_time_mean', 'update2create_time_mean', 'send2update_time_mean',\
               'overall_time_min', 'update2create_time_min', 'send2update_time_min',\
               'overall_time_max', 'update2create_time_max', 'send2update_time_max']

In [32]:
data[pd.isnull(data['overall_time_mean'])]

,order_sd,active_days,lastgap,avg_gap,min_gap,max_gap,total_orders,w1_total_orders,w2_total_orders,w3_total_orders,...,send2update_time_max,intime_rate,last_coupon_fee,last_send_type,order_nums,city_id,sex,age,login_platform_b,app_version_b


In [33]:
data[pd.isnull(data['overall_time_min'])]

,order_sd,active_days,lastgap,avg_gap,min_gap,max_gap,total_orders,w1_total_orders,w2_total_orders,w3_total_orders,...,send2update_time_max,intime_rate,last_coupon_fee,last_send_type,order_nums,city_id,sex,age,login_platform_b,app_version_b


In [35]:
data[pd.isnull(data['overall_time_max'])]

,order_sd,active_days,lastgap,avg_gap,min_gap,max_gap,total_orders,w1_total_orders,w2_total_orders,w3_total_orders,...,send2update_time_max,intime_rate,last_coupon_fee,last_send_type,order_nums,city_id,sex,age,login_platform_b,app_version_b


In [34]:
for f in time_features:
    data[f]=data[f].fillna(500000)

In [31]:
# 9.intime_rate fillna 赋0
data['intime_rate']=data['intime_rate'].fillna(0)

In [32]:
# 整合，看新data的样子什么的！

In [33]:
# this piece of code from one kaggle
# 看看有没有NAN什么的,Type Error说明没有NAN了！
import matplotlib.pyplot as plt
nan_value = data.isnull().sum()
nan_value = nan_value[nan_value > 0]
nan_value.sort_values(inplace=True,ascending=False)
nan_value.plot.bar()

TypeError: Empty 'DataFrame': no numeric data to plot

In [34]:
data.to_csv('data_cleaned.csv',index=False)

In [36]:
data=pd.read_csv('data_cleaned.csv')

In [37]:
data.columns

Index(['order_sd', 'active_days', 'lastgap', 'avg_gap', 'min_gap', 'max_gap',
       'total_orders', 'w1_total_orders', 'w2_total_orders', 'w3_total_orders',
       'w4_total_orders', 'w5_total_orders', 'w6_total_orders',
       'w7_total_orders', 'total_couponworth', 'w1_total_couponworth',
       'w2_total_couponworth', 'w3_total_couponworth', 'w4_total_couponworth',
       'w5_total_couponworth', 'w6_total_couponworth', 'w7_total_couponworth',
       'label', 'first_order', 'last_order', 'accountLength',
       'overall_time_mean', 'update2create_time_mean', 'send2update_time_mean',
       'overall_time_min', 'update2create_time_min', 'send2update_time_min',
       'overall_time_max', 'update2create_time_max', 'send2update_time_max',
       'intime_rate', 'last_coupon_fee', 'last_send_type', 'order_nums',
       'city_id', 'sex', 'age', 'login_platform_b', 'app_version_b'],
      dtype='object')

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191855 entries, 0 to 191854
Data columns (total 44 columns):
order_sd                   191855 non-null float64
active_days                191855 non-null int64
lastgap                    191855 non-null float64
avg_gap                    191855 non-null float64
min_gap                    191855 non-null float64
max_gap                    191855 non-null float64
total_orders               191855 non-null float64
w1_total_orders            191855 non-null float64
w2_total_orders            191855 non-null float64
w3_total_orders            191855 non-null float64
w4_total_orders            191855 non-null float64
w5_total_orders            191855 non-null float64
w6_total_orders            191855 non-null float64
w7_total_orders            191855 non-null float64
total_couponworth          191855 non-null float64
w1_total_couponworth       191855 non-null float64
w2_total_couponworth       191855 non-null float64
w3_total_couponworth    

In [39]:
len(data)

191855